In [1]:
from attalos.imgtxt_algorithms.main import *

In [2]:
# Temp object using duck typing to replace command line arguments
args = lambda: None

"""args.image_feature_file_train = "/local_data/teams/attalos/features/image/yfcc0_train_20160831_inception.hdf5"
args.text_feature_file_train = "/local_data/teams/attalos/features/text/yfcc0_train_20160831_text.json.gz"
args.image_feature_file_test = "/local_data/teams/attalos/features/image/yfcc0_train_20160831_inception.hdf5"
args.text_feature_file_test = "/local_data/teams/attalos/features/text/yfcc0_train_20160831_text.json.gz"
"""

args.image_feature_file_train = "/local_data/teams/attalos/features/image/iaprtc_train_20160816_inception.hdf5"
args.text_feature_file_train = "/local_data/teams/attalos/features/text/iaprtc_train_20160816_text.json.gz"
args.image_feature_file_test = "/local_data/teams/attalos/features/image/iaprtc_test_20160816_inception.hdf5"
args.text_feature_file_test = "/local_data/teams/attalos/features/text/iaprtc_test_20160816_text.json.gz"

#args.word_vector_file = "/data/fs4/teams/attalos/wordvecs/text9Bvin.bin"
args.word_vector_file = "/local_data/kni/data/vectors-phrase.bin"
args.word_vector_type = "w2v" # "glove"

args.model_type = "yfccjoint" #"negsampling" #"wdv" #"naivesum" #"multihot"
args.cross_eval = False
args.in_memory = True
args.model_input_path = None
args.model_output_path = None
args.num_epochs = 5000
args.batch_size = 100
args.learning_rate = 0.0001
args.verbose_eval = True
args.epoch_verbosity = 10
args.optim_words = True

In [3]:
logger.info("Parsing train and test datasets.")
train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
test_dataset = train_dataset
#test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test, load_image_feats_in_mem=args.in_memory)

[2016-09-23 19:54:28,131] [INFO] Parsing train and test datasets.
[2016-09-23 19:54:28,132] [INFO] Loading image features...
[2016-09-23 19:54:28,133] [INFO] Reading h5py file.
[2016-09-23 19:54:28,134] [INFO] Loading image feats.
[2016-09-23 19:54:28,250] [INFO] Elapsed time: 0.116657018661
[2016-09-23 19:54:28,251] [INFO] Getting length of first feature vector.
[2016-09-23 19:54:28,252] [INFO] Num images: 17665
[2016-09-23 19:54:28,252] [INFO] Loading text features...
[2016-09-23 19:54:28,253] [INFO] Loading json from file (timing).
[2016-09-23 19:54:28,407] [INFO] Time elapsed: 0.1537129879


In [4]:
logger.info("Reading word vectors from file.")
wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)

[2016-09-23 19:54:28,412] [INFO] Reading word vectors from file.


In [5]:
#sess.close()

---

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [7]:
model_cls = ModelTypes[args.model_type].value
logger.info("Selecting model class: %s" % model_cls.__name__)
datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
model = model_cls(wv_model, datasets, **vars(args))
model.initialize_model(sess)

[2016-09-23 19:54:41,188] [INFO] Selecting model class: YFCCJointNegSamplingModel
[2016-09-23 19:54:42,364] [INFO] Constructing model info.
[2016-09-23 19:54:42,369] [INFO] Optimization on GPU, word vectors are stored separately.


In [8]:
logger.info("Preparing test_dataset.")
fetches, feed_dict, truth = model.prep_predict(test_dataset)

[2016-09-23 19:54:43,235] [INFO] Preparing test_dataset.
[2016-09-23 19:54:43,355] [INFO] Allocating space for text feature matrix.
[2016-09-23 19:54:43,364] [INFO] Done allocating.
[2016-09-23 19:54:43,364] [INFO] Starting prep_predict loop.
[2016-09-23 19:54:43,646] [INFO] 0 of 17665 completed (prep_predict).


In [9]:
logger.info("Starting training phase.")
mid_eval = (fetches, feed_dict, truth) if args.verbose_eval else None
train(sess, model, args.num_epochs, train_dataset, args.batch_size, args.epoch_verbosity, mid_eval=mid_eval) #, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)

[2016-09-23 19:54:44,299] [INFO] Starting training phase.
[2016-09-23 19:54:47,407] [DEBUG] Finished epoch 0. (Avg. training loss: 1.19261980937)
[2016-09-23 19:54:48,192] [DEBUG] Evaluation (precision, recall, f1): [0.13823898586248207, 0.14938609436405936, 0.10713585962277707]
[2016-09-23 19:55:17,127] [DEBUG] Finished epoch 10. (Avg. training loss: 0.841832520948)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[2016-09-23 19:55:17,913] [DEBUG] Evaluation (precision, recall, f1): [0.29557892328204438, 0.26844862185070489, 0.18618369974531651]
[2016-

In [10]:
logger.info("Starting evaluation phase.")
evaluate(sess, model, fetches, feed_dict, truth)

[2016-09-23 20:19:07,304] [INFO] Starting evaluation phase.
[2016-09-23 20:19:08,019] [DEBUG] Evaluation (precision, recall, f1): [0.41943290284567158, 0.48279578181373967, 0.39634839328708843]


[0.41943290284567158, 0.48279578181373967, 0.39634839328708843]

In [11]:
print """ pre-allocate, for-loop on text features
[2016-09-20 18:19:06,785] [INFO] Preparing test_dataset.
[2016-09-20 18:19:06,786] [INFO] Allocating space for matrices.
[2016-09-20 18:19:38,713] [INFO] Done allocating.
[2016-09-20 18:19:38,715] [INFO] Starting prep_predict loop.
[2016-09-20 18:19:38,771] [INFO] 0 of 5052092 completed (prep_predict).
[2016-09-20 18:20:45,936] [INFO] 500000 of 5052092 completed (prep_predict).
[2016-09-20 18:21:52,126] [INFO] 1000000 of 5052092 completed (prep_predict).
[2016-09-20 18:23:00,177] [INFO] 1500000 of 5052092 completed (prep_predict).
[2016-09-20 18:24:07,410] [INFO] 2000000 of 5052092 completed (prep_predict).
[2016-09-20 18:25:14,794] [INFO] 2500000 of 5052092 completed (prep_predict).
[2016-09-20 18:26:21,566] [INFO] 3000000 of 5052092 completed (prep_predict).
[2016-09-20 18:27:28,465] [INFO] 3500000 of 5052092 completed (prep_predict).
[2016-09-20 18:28:48,673] [INFO] 4000000 of 5052092 completed (prep_predict).
[2016-09-20 18:30:54,054] [INFO] 4500000 of 5052092 completed (prep_predict).
[2016-09-20 18:33:12,869] [INFO] 5000000 of 5052092 completed (prep_predict).


"""

 pre-allocate, for-loop on text features
[2016-09-20 18:19:06,785] [INFO] Preparing test_dataset.
[2016-09-20 18:19:06,786] [INFO] Allocating space for matrices.
[2016-09-20 18:19:38,713] [INFO] Done allocating.
[2016-09-20 18:19:38,715] [INFO] Starting prep_predict loop.
[2016-09-20 18:19:38,771] [INFO] 0 of 5052092 completed (prep_predict).
[2016-09-20 18:20:45,936] [INFO] 500000 of 5052092 completed (prep_predict).
[2016-09-20 18:21:52,126] [INFO] 1000000 of 5052092 completed (prep_predict).
[2016-09-20 18:23:00,177] [INFO] 1500000 of 5052092 completed (prep_predict).
[2016-09-20 18:24:07,410] [INFO] 2000000 of 5052092 completed (prep_predict).
[2016-09-20 18:25:14,794] [INFO] 2500000 of 5052092 completed (prep_predict).
[2016-09-20 18:26:21,566] [INFO] 3000000 of 5052092 completed (prep_predict).
[2016-09-20 18:27:28,465] [INFO] 3500000 of 5052092 completed (prep_predict).
[2016-09-20 18:28:48,673] [INFO] 4000000 of 5052092 completed (prep_predict).
[2016-09-20 18:30:54,054] [INFO]

In [12]:
print """time comparison:
loading driver=None:
[2016-09-19 19:36:15,041] [INFO] Parsing train and test datasets.
[2016-09-19 19:36:15,042] [INFO] Loading image features...
[2016-09-19 19:36:15,043] [INFO] Reading h5py file.
[2016-09-19 19:36:15,044] [INFO] Loading image feats.
[2016-09-19 19:39:46,185] [INFO] Elapsed time: 211.141229868
[2016-09-19 19:39:46,187] [INFO] Getting length of first feature vector.
[2016-09-19 19:39:46,188] [INFO] Num images: 9748229
[2016-09-19 19:39:46,188] [INFO] Loading text features...
[2016-09-19 19:39:46,189] [INFO] Loading json from file (timing).
[2016-09-19 19:40:41,631] [INFO] Time elapsed: 55.441298008

[2016-09-19 20:58:09,687] [INFO] Parsing train and test datasets.
[2016-09-19 20:58:09,688] [INFO] Loading image features...
[2016-09-19 20:58:09,689] [INFO] Reading h5py file.
[2016-09-19 20:58:09,690] [INFO] Loading image feats.
[2016-09-19 21:08:34,432] [INFO] Elapsed time: 624.742087841
[2016-09-19 21:08:34,434] [INFO] Getting length of first feature vector.
[2016-09-19 21:08:34,435] [INFO] Num images: 9748229
[2016-09-19 21:08:34,436] [INFO] Loading text features...
[2016-09-19 21:08:34,436] [INFO] Loading json from file (timing).
[2016-09-19 21:09:40,966] [INFO] Time elapsed: 66.5285241604

inception
[2016-09-19 21:55:28,575] [INFO] Parsing train and test datasets.
[2016-09-19 21:55:28,576] [INFO] Loading image features...
[2016-09-19 21:55:28,576] [INFO] Reading h5py file.
[2016-09-19 21:55:28,578] [INFO] Loading image feats.
[2016-09-19 22:02:41,962] [INFO] Elapsed time: 433.384609938
[2016-09-19 22:02:41,965] [INFO] Getting length of first feature vector.
[2016-09-19 22:02:41,966] [INFO] Num images: 5052092
[2016-09-19 22:02:41,967] [INFO] Loading text features...
[2016-09-19 22:02:41,967] [INFO] Loading json from file (timing).
[2016-09-19 22:07:11,417] [INFO] Time elapsed: 269.449061871

[2016-09-20 17:04:07,657] [INFO] Parsing train and test datasets.
[2016-09-20 17:04:07,658] [INFO] Loading image features...
[2016-09-20 17:04:07,658] [INFO] Reading h5py file.
[2016-09-20 17:04:07,661] [INFO] Loading image feats.
[2016-09-20 17:05:40,185] [INFO] Elapsed time: 92.5259859562
[2016-09-20 17:05:40,187] [INFO] Getting length of first feature vector.
[2016-09-20 17:05:40,187] [INFO] Num images: 5052092
[2016-09-20 17:05:40,188] [INFO] Loading text features...
[2016-09-20 17:05:40,189] [INFO] Loading json from file (timing).

"""

time comparison:
loading driver=None:
[2016-09-19 19:36:15,041] [INFO] Parsing train and test datasets.
[2016-09-19 19:36:15,042] [INFO] Loading image features...
[2016-09-19 19:36:15,043] [INFO] Reading h5py file.
[2016-09-19 19:36:15,044] [INFO] Loading image feats.
[2016-09-19 19:39:46,185] [INFO] Elapsed time: 211.141229868
[2016-09-19 19:39:46,187] [INFO] Getting length of first feature vector.
[2016-09-19 19:39:46,188] [INFO] Num images: 9748229
[2016-09-19 19:39:46,188] [INFO] Loading text features...
[2016-09-19 19:39:46,189] [INFO] Loading json from file (timing).
[2016-09-19 19:40:41,631] [INFO] Time elapsed: 55.441298008

[2016-09-19 20:58:09,687] [INFO] Parsing train and test datasets.
[2016-09-19 20:58:09,688] [INFO] Loading image features...
[2016-09-19 20:58:09,689] [INFO] Reading h5py file.
[2016-09-19 20:58:09,690] [INFO] Loading image feats.
[2016-09-19 21:08:34,432] [INFO] Elapsed time: 624.742087841
[2016-09-19 21:08:34,434] [INFO] Getting length of first feature ve